In [48]:
import json
import pandas as pd
import sys

f = open('routes.json')
routes = json.load(f)
routes_df = pd.DataFrame.from_records(routes.get('routes'))

f = open('vehicles.json')
vehicles = json.load(f)
vehicles_df = pd.DataFrame.from_records(vehicles.get('vehicles'))

# for each vehicle, need range
# for each route, need total_distance
# for each route - vehicle pair s.t. r.total_distance <= v.range
#   need energy_consumed (r.total_distance/100 * v.kwh_per_100_km)

for index, v in vehicles_df.iterrows():
    try:
        v_range = (v['capacity_kwh'] / v['kwh_per_100_km']) * 100
    except:
        print("Missing information about vehicle {}: Cannot calculate maximum distance".format(v['id']))
        v_range = 0
    vehicles_df.loc[vehicles_df.index[index], 'range'] = v_range

print(vehicles_df)

     id  capacity_kwh  kwh_per_100_km      range
0  v001           3.0              15  20.000000
1  v006           7.0              17  41.176471
2  v004           7.0              17  41.176471
3  v009          16.0              20  80.000000
4  v005           8.0              17  47.058824
5  v007           NaN              21        NaN
6  v002           5.0              16  31.250000
7  v003           4.0              15  26.666667
8  v008          12.0              15  80.000000


In [49]:
possible_vehicles = []
for index, r in routes_df.iterrows():
    distance = 0
    try:
        stops = r['stops']
        for s in stops:
            distance += s['distance_km']
    except:
        print("Missing information about route {}: Cannot calculate maximum distance".format(r['route_id']))
    routes_df.loc[routes_df.index[index], 'total_distance'] = distance
        
    eligible_vehicles = {}
    for _, v in vehicles_df.iterrows():
        try:
            if distance <= v['range']:
                energy_consumed = round((distance/100) * v['kwh_per_100_km'], 2)
                eligible_vehicles.update({v['id']: energy_consumed})
        except:
            print("Unable to calculate energy consumed for route {} with vehicle {}".format(r['route_id'],v['id']))
            
    possible_vehicles.append(eligible_vehicles)
    print(eligible_vehicles)
#     routes_df.loc[routes_df.index[index], 'eligible_vehicles'] = eligible_vehicles

routes_df.loc[:, 'eligible_vehicles'] = possible_vehicles
print(routes_df)

# NB: stops aren't unique

{'v001': 3.0, 'v006': 3.4, 'v004': 3.4, 'v009': 4.0, 'v005': 3.4, 'v002': 3.2, 'v003': 3.0, 'v008': 3.0}
{'v006': 5.86, 'v004': 5.86, 'v009': 6.9, 'v005': 5.86, 'v008': 5.17}
{'v001': 2.62, 'v006': 2.97, 'v004': 2.97, 'v009': 3.5, 'v005': 2.97, 'v002': 2.8, 'v003': 2.62, 'v008': 2.62}
{'v006': 5.51, 'v004': 5.51, 'v009': 6.48, 'v005': 5.51, 'v008': 4.86}
{'v006': 4.61, 'v004': 4.61, 'v009': 5.42, 'v005': 4.61, 'v002': 4.34, 'v008': 4.07}
{'v009': 8.98, 'v005': 7.63, 'v008': 6.74}
{'v009': 10.18, 'v008': 7.63}
{'v009': 10.54, 'v008': 7.91}
   route_id                                              stops  \
0  R2731820  [{'stop_id': 'a441f0e8-77cc-41e7-bc67-14a3a0b8...   
1  R5839156  [{'stop_id': 'd69c6f05-85b3-4e84-a3e8-eb2b9d9e...   
2  R7527039  [{'stop_id': 'a89089ab-24c2-449e-9f34-df0de29f...   
3  R4950482  [{'stop_id': 'aaff3909-880e-428d-81d5-5fa3f5af...   
4  R8370118  [{'stop_id': 'd8f928fe-7f25-4c29-b0af-8c8f9a1a...   
5  R3102246  [{'stop_id': 'e2f2f764-3129-4a28-bc8f-b93e4b4f

In [50]:
# Assuming the average electricity consumption value is accurate / an upper estimate 
#   and we're okay with the vehicles potentially having exactly 0 power left after the route

In [51]:
# The list of optimal vehicle-route pairs
# Assuming this refers to the best vehicles for each route, regardless of duplication

optimal_pairs = {}
for _, r in routes_df.iterrows():
    out = min(r['eligible_vehicles'], key=lambda x: x[1])
    optimal_pairs.update({r['route_id']: out})
    
for route, vehicle in optimal_pairs.items():
    print("Route {}: Vehicle {}".format(route, vehicle))
    
# What if more than one is best?

Route R2731820: Vehicle v001
Route R5839156: Vehicle v006
Route R7527039: Vehicle v001
Route R4950482: Vehicle v006
Route R8370118: Vehicle v006
Route R3102246: Vehicle v009
Route R6660789: Vehicle v009
Route R4673799: Vehicle v009


In [52]:
# The total kWh required to complete all routes sequentially using the least efficient vehicle only
# "least efficient vehicle" - one vehicle per route or one vehicle overall?
# "Sequentially" = duplication of vehicles allowed

total_energy = 0
for _, r in routes_df.iterrows():
    out = max(r['eligible_vehicles'].items(), key=lambda x: x[1])
    print("Vehicle {} assigned to route {}".format(out[0], r['route_id']))
    total_energy += out[1]
print("Total energy consumed when using the least efficient vehicles for each route: {}kWh".format(total_energy))

# Oh ended up being a single vehicle anyway

Vehicle v009 assigned to route R2731820
Vehicle v009 assigned to route R5839156
Vehicle v009 assigned to route R7527039
Vehicle v009 assigned to route R4950482
Vehicle v009 assigned to route R8370118
Vehicle v009 assigned to route R3102246
Vehicle v009 assigned to route R6660789
Vehicle v009 assigned to route R4673799
Total energy consumed when using the least efficient vehicles for each route: 56.0kWh


In [53]:
# The total kWh required to complete all routes in parallel using the optimal vehicle-route pairs
# "In parallel" = not enough info to determine how fast each vehicle goes, i.e. how long each journey is
# So assume no duplication of vehicles at all: 8 routes and 8 (usable) vehicles

# -> essentially a matching problem
# Minimise energy
# Where vi and ri have an edge between them if vi is in ri.eligible_vehicles[0] weighted ri.eligible_vehicles[1]

# Simplest approach: Greedy


route_vehicle_pairs = {}
for _, r in routes_df.iterrows():
    for v, e in r['eligible_vehicles'].items():
        route_vehicle_pairs[(r['route_id'], v)] = e

route_vehicle_pairs = dict(sorted(route_vehicle_pairs.items(), key=lambda x: x[1]))

total_energy = 0
used_routes = []
used_vehicles = []
for x, w in route_vehicle_pairs.items():
    if (x[0] not in used_routes) and (x[1] not in used_vehicles):
        print(x)
        used_routes.append(x[0])
        used_vehicles.append(x[1])
        total_energy += w
    
print("Total energy consumed: {}kWh".format(total_energy))

# Doesn't account for dead ends

('R7527039', 'v001')
('R2731820', 'v003')
('R8370118', 'v008')
('R4950482', 'v006')
('R5839156', 'v004')
('R3102246', 'v005')
('R6660789', 'v009')
Total energy consumed: 38.870000000000005kWh


In [54]:
# As a Graph problem:
# nodes are either routes or vehicles, edges are possible matches weighted with energy_consumed
# construct graph then remove maximal edges to end with minimal matches
# making a residual graph s.t. each node is visited exactly once and the total weight is minimised
# -> shortest path in a complete weighted digraph (sounds NP-Hard?)

# Can optimise it a bit at least, start with routes with least available vehicles
# And sort a route_node's neighbours for greedy approach?


class Node:
    def __init__(self, key):
        self.key = key
        self.neighbours =  {} # neighbour node : weight
        self.visited = False
        
    def add_neighbour(self, neighbour, weight):
        self.neighbours[neighbour] = weight
    
#     def sort_edges(self):
#         sorted_edges = sorted(self.neighbours.items(), key=lambda x: x[1])
#         self.neighbours = dict(sorted_edges)

class Graph:
    def __init__(self, G = {}):
        self.nodes = {}

    def add_edge(self, r, v, w): 
        if r not in self.nodes:
            self.nodes[r] = Node(r)
        self.nodes[r].add_neighbour(v, w)
        
    def sort_everything(self):
        sorted_nodes = sorted(self.nodes.items(), key=lambda x: len(x[1].neighbours))
        self.nodes = dict(sorted_nodes)
#         for r in self.nodes.items():
#             r[1].sort_edges()
            
G = Graph()
for _, r in routes_df.iterrows():
    for v, e in r['eligible_vehicles'].items():
        G.add_edge(r['route_id'], v, e)
G.sort_everything()

route_nodes_only = G.nodes.copy()
for _, v in vehicles_df.iterrows(): #each vehicle also needs an edge to every route with weight 0
    for r in route_nodes_only.items(): # iterate over current copy (only route nodes)
        G.add_edge(v['id'], r[0], 0)

# for x in G.nodes.items():
#     print(x[0])
#     print(x[1].neighbours.items())
    
total_energy = 0

def dfs(G, node, path, weight):
    path += [node]
    G.nodes[node].visited = True
    if len(path) == len(route_nodes_only)*2:
        print(path)
        print("Total energy consumed: {}kWh".format(weight))
    for n, w in G.nodes[node].neighbours.items():
#         print("node {} to node {}".format(node, n))
        if not G.nodes[n].visited and n not in path:
#             print("using path {} to {} with weight {}".format(node, n, w))
            weight += w
            dfs(G, n, path, weight)

src = list(G.nodes.items())[0][0]
weight = dfs(G, src, [], 0)

# Not optimal (see greedy solution), but now at least won't get stuck in dead ends

['R6660789', 'v009', 'R4673799', 'v008', 'R3102246', 'v005', 'R5839156', 'v006', 'R4950482', 'v004', 'R8370118', 'v002', 'R2731820', 'v001', 'R7527039', 'v003']
Total energy consumed: 47.04999999999999kWh
